In [ ]:
import pandas as pd
import numpy as np

### Pregunta 1

In [ ]:
movies = pd.read_csv('movies.csv', encoding = "ISO-8859-1")
variables = movies.select_dtypes(include=np.number)

normalized_variables =(variables-variables.min())/(variables.max()-variables.min())
print(normalized_variables)